<a href="https://colab.research.google.com/github/serciex/lane-change/blob/main/v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Prerequisites

In [ ]:
!pip install pygame
!pip install highway-env

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 10.9 MB/s eta 0:00:00


Testing

In [ ]:
#Import Libraries
from os import truncate
import math
import gymnasium
import highway_env
from matplotlib import pyplot as plt
import pygame
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
import gym
from random import randint
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from collections import deque
import random
import torch.optim as optim
import tqdm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from IPython.display import display


# Workaround for gym compatibility
if not hasattr(np, 'bool8'):
    np.bool8 = np.bool_

%matplotlib inline

def animate_image_list_efficient(image_list):
    fig, ax = plt.subplots(figsize=(10, 6))

    # Create initial image
    im = ax.imshow(image_list[0])
    ax.axis('off')
    title = ax.set_title('Frame 0')

    def animate(frame):
        # Just update the image data - more efficient
        im.set_array(image_list[frame])
        title.set_text(f'Frame {frame}/{len(image_list)}')
        return [im, title]

    anim = FuncAnimation(fig, animate, frames=len(image_list),
                        interval=100, blit=True, repeat=True)
    plt.close()
    return HTML(anim.to_jshtml())

# Initialize plot outside the loop (before the training loop starts)
plt.figure(figsize=(10, 6))

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

Environment Definition

In [ ]:
# Configure Environment Conditions
config = {
    "lanes_count": 3,
    "lane_width": 3.75,
    "observation": {
        "type": "Kinematics",
        "features": ["x", "y", "vx", "vy", "heading", "lat_off"]
    },
    "action": {"type": "ContinuousAction"},"ego_spawn_random": True,
    "policy_frequency": 10,
}
env = gymnasium.make('highway-v0', render_mode='rgb_array', config=config)
frames = []

# Action Setup
highway_env.envs.common.action.ContinuousAction(env, lateral=True,
                                                longitudinal=True)

Environment Wrapper

In [ ]:
(env.unwrapped.vehicle.velocity[0])

np.float64(22.879332711210857)

In [ ]:
class ENVwrapper(env.wrapper):
  def __init__(self,env, w1,w2,w3,parameters):
    super().__init__(env)
    self.w1 = w1
    self.w2 = w2
    self.w3 = w3
    self.desired_parameters,self.control_parameters, self.reward_weights, self.following_gap = parameters

    def lane_check(self):
      """
      Determines which adjacent lane offers the largest front gap for a lane change.
      """
      obs = self.env.observation
      ego = env.unwrapped.vehicle
      current_lane_id = env.unwrapped.vehicle.lane_index[2]

      # Reset flag when lane changes
      if self.last_lane is not None and current_lane_id != self.last_lane:
          self.gap_controller_checked = False

      # Only check if not already checked
      if not self.gap_controller_checked:

          # Get front gaps using existing longitudinal_lead_state
          gap_current_front = obs[1][0]

          # Get right lane front gap
          current_lane = list(ego.lane_index)
          lane_right = (current_lane[0], current_lane[1], current_lane[2] + 1) if current_lane[2] < 2 else ego.lane_index
          neighbours_right = env.unwrapped.road.neighbour_vehicles(ego, lane_index=lane_right)
          gap_right_front = neighbours_right[0].position[0] - ego.position[0] if neighbours_right and neighbours_right[0] else -float('inf')

          # Get left lane front gap
          lane_left = (current_lane[0], current_lane[1], current_lane[2] - 1) if current_lane[2] > 0 else ego.lane_index
          neighbours_left = env.unwrapped.road.neighbour_vehicles(ego, lane_index=lane_left)
          gap_left_front = neighbours_left[0].position[0] - ego.position[0] if neighbours_left and neighbours_left[0] else -float('inf')

          # Find best lane
          front_gaps = [gap_current_front, gap_right_front, gap_left_front]
          best_index = np.argmax(front_gaps)

          if best_index == 1:
              direction = 1  # Right
              gap_right_follower = ego.position[0] - neighbours_right[1].position[0] if len(neighbours_right) > 1 and neighbours_right[1] else float('inf')
              self.target_id = direction if gap_right_follower >= self.following_gap_threshold else 0
          elif best_index == 2:
              direction = -1  # Left
              gap_left_follower = ego.position[0] - neighbours_left[1].position[0] if len(neighbours_left) > 1 and neighbours_left[1] else float('inf')
              self.target_id = direction if gap_left_follower >= self.following_gap_threshold else 0
          else:
              self.target_id = 0  # Stay in current lane

          self.gap_controller_checked = True

      self.last_lane = current_lane_id

      return target_id

    def idm_controller(self):
      obs = self.env.observation
      s0,v0,epsilon = self.desired_parameters
      a,b,delta,T = self.control_parameters

      v = obs[0][3]
      delta_v = obs[0][4]
      s = obs[0][5]

      # Desired gap: s* = s0 + v*T + (v * delta_v) / (2 * sqrt(a * b))
      desired_gap = self.s0 + max(0, v * self.T + ((v * delta_v) / (2 * math.sqrt(self.a * self.b))))

      # IDM acceleration: a_IDM = a * [ 1 - (v / v0)^delta - (s* / s)^2 ]
      acceleration = self.a * (1 - (v / self.v0)**self.delta - (desired_gap / (s + epsilon))**2)

      return acceleration

    def reward_function(self, obs_old, obs_new):
      target_id = self.lateral_target_id()
      w1,w2,w3 = self.reward_weights
      obs_new = self.env.observation
      obs_old = self.env.observation_old

      """
      Reward Function:

      Acceleration Reward: r_acce = w1*f_acce(a_yaw)
      a_yaw = yaw acceleration (rate of change of yaw rate)

      Rate Reward: r_rate = w2*f_rate(w_yaw)
      w_yaw = yaw rate (rate of change of heading)

      Time Reward: r_time = w3*f_time (delta_lat_deviation)
      delta_lat_deviation = change in lateral deviation (self.lat_off)

      Reward = Cummulative Sum of r_acce + Cummulative Sum of r_rate + Cummulative Sum of r_time

      """

      # Convert direction command to actual target lane
      current_lane_id = env.unwrapped.vehicle.lane_index[2]

      if target_id == +1:  # Move right
          actual_target_lane = min(2, current_lane_id + 1)  # Don't exceed rightmost lane
      elif target_id == -1:  # Move left
          actual_target_lane = max(0, current_lane_id - 1)  # Don't exceed leftmost lane
      else:  # target_id == 0, stay in current lane
          actual_target_lane = current_lane_id

      # Use current vehicle lane structure and set actual target lane
      current_lane_index = list(env.unwrapped.vehicle.lane_index)
      current_lane_index[2] = actual_target_lane  # Only change the lane number to actual lane
      self.target_lane = tuple(current_lane_index)

      target_lane_object = env.unwrapped.road.network.get_lane(self.target_lane)
      vehicle_s, _ = env.unwrapped.vehicle.lane.local_coordinates(env.unwrapped.vehicle.position)
      _ , self.delta_lat_deviaton = target_lane_object.local_coordinates(env.unwrapped.vehicle.position)

      obs = obs_new[0]
      obs_old = obs_old[0]

      # Extract lateral and longitudinal velocities from observations
      vx_current = obs[2]  # longitudinal velocity
      vy_current = obs[3]  # lateral velocity
      vx_old = obs_old[2]
      vy_old = obs_old[3]

      dt = 1.0 / env.unwrapped.config['policy_frequency']
      L = env.unwrapped.vehicle.LENGTH

      # Calculate both yaw rates in parallel using bicycle model
      vx_vals = np.array([vx_old, vx_current])
      vy_vals = np.array([vy_old, vy_current])

      # Vectorized calculations
      total_velocities = np.sqrt(vx_vals**2 + vy_vals**2)
      curvatures = np.divide(vy_vals, vx_vals * total_velocities + 1e-6,
                            out=np.zeros_like(vy_vals), where=(abs(vx_vals) > 1e-6))
      yaw_rates = total_velocities * curvatures

      previous_yaw_rate, current_yaw_rate = yaw_rates[0], yaw_rates[1]

      w_yaw = current_yaw_rate
      w_yaw_old = previous_yaw_rate

      self.w_acce = (w_yaw-w_yaw_old)*env.unwrapped.config['policy_frequency']

      # Acceleration Reward
      acce_reward = -w1*abs(self.w_acce)

      # Rate Reward
      rate_reward = -w2*abs(w_yaw)

      # Time Reward
      time_reward = -w3*((self.delta_lat_deviaton)**2)

      # Off Road
      if not env.unwrapped.vehicle.on_road:
          off_road_penalty = -10

      else:
          off_road_penalty = 0

      # Overall Reward
      self.reward = acce_reward + rate_reward + time_reward + off_road_penalty

      return [self.reward, acce_reward, rate_reward, time_reward, off_road_penalty]


    def step(self, agent_action):
      vehicle = env.unwrapped.vehicle
      idm_action = self.idm_controller()
      action = [idm_action, agent_action]

      # Prior Observation
      obs_old = env.observation[0]
      v_prev = obs_old[0][2]

      obs, reward, done, truncated, info = super().step(action)
      custom_reward = self.reward_function(obs_old,obs)

      done = done or truncated if vehicle.on_road and not info['crashed'] else True

      x = obs[0][0]
      y = obs[0][1]
      vx = obs[0][2]
      thetha = obs[0][4]
      lane_width = vehicle.lane.width

      lane_id = self.lane_check()

      self_curvature = vehicle.lane.heading_at(np.clip(
          vehicle.lane.local_coordinates(vehicle.position)[0],
          0, vehicle.lane.length))

      acceleration = (vx-v_prev)*env.unwrapped.config['policy_frequency']

      agent_observation = [x,y,vx,acceleration,thetha,lane_id,lane_width,self_curvature]

      return agent_observation, custom_reward, done, truncated, info

      def


Agent Defintion

In [ ]:
class ActorCritic(nn.Module):
    def __init__(self, state_dim=8, action_dim=1, hidden_size=128, max_angle=0.5):
        super().__init__()
        self.action_dim = action_dim
        self.max_angle = max_angle

        # Policy head (actor) - Output 2 values: mean and log_std
        self.shared = nn.Sequential(
            nn.Linear(state_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU()
        )

        self.mu = nn.Sequential(
            nn.Linear(hidden_size, action_dim)
                                )
        self.sigma = nn.Sequential(
            nn.Linear(hidden_size, action_dim),
                                   nn.Softplus()
                                   )

        # Value head (critic)
        self.critic = nn.Sequential(
            nn.Linear(state_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )

    def forward(self, x):
        # Value head
        value = self.critic(x)

        # Policy head
        actor_output = self.shared(x)

        action_mean = (self.mu(actor_output))*self.max_angle

        action_std = self.sigma(actor_output)
        action_std = torch.clamp(action_std, -3, 3)

        # Action Distribution
        action_dist = torch.distributions.Normal(action_mean, action_std)

        return action_dist, value

    def action(self, state, deterministic = False):
        action_dist, value = self.forward(state)

        if deterministic:
            action = action_dist.mean
            log_prob = action_dist.log_prob(action)

        else:
          action = action_dist.sample()
          log_prob = action_dist.log_prob(action)

        return action, log_prob, value

Lateral Controller (Gap Checker)

Initialization

In [ ]:
################################################################################
"""Initialization"""
################################################################################
# Environment
# Initialize Environment
obs, _ = env.reset()

# Agent Initialization
total_steps_taken = 0

# Tracking for performance
timestep_rewards = []  # Store rewards by timestep
timestep_acce_rewards = []  # Store acceleration rewards
timestep_rate_rewards = []  # Store rate rewards
timestep_time_rewards = []  # Store time rewards
global_returns = []  # Cumulative return tracking
loss_history = []  # Loss values
timesteps_list = []  # Track timesteps for plotting
episode_lengths = []  # Track episode lengths
episode_indices = []  # Track episode indices for plotting

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# IDM Parameters
desired_parameters = [20, 10.0]       # s0, v0
control_parameters = [0.1, 5, 4, 4]  # a, b, δ, T

# Initialize Environment Manager
state_manager = ENV(obs, desired_parameters, control_parameters)

# Get initial target and state info
target_id = state_manager.lane_checker()
agent_state = state_manager.ego_state_agent(target_id=target_id)
state_dim = len(agent_state)
action_dim = 1

# Environment Information
L = env.unwrapped.vehicle.LENGTH

# Initialize reward and states
state_manager.reward_function(obs, obs, target_id, w1=1, w2=1, w3=0.05)
ego_state_idm = state_manager.ego_state_idm()
lead_state = state_manager.longitudinal_lead_state()

Model

Hyperparameters

In [ ]:
# Define hyperparameters
max_timesteps_per_rollout = 500  # Max timesteps per rollout
K = 10    # Number of policy update epochs
test_rate = 5  # Test every 5 batches
rollout_size = 5

batch_size = 32
gamma = 0.99
vf_coeff_cl = 0.5
ent_coef_c2 = 0.01
epsilon = 0.2
gae_lambda = 0.99
actor_lr = 3e-4  # Reduced learning rate
critic_lr = 1e-3
hidden_size = 256

# Episode counter
max_episodes_per_rollout = 1
nb_episodes = 1000
nb_batches = round(nb_episodes/max_episodes_per_rollout)

# Initialize model and optimizer
model = ActorCritic(state_dim=state_dim, action_dim=action_dim,hidden_size=hidden_size).to(device)
actor_params = list(model.shared.parameters()) + list(model.mu.parameters()) + list(model.sigma.parameters())
critic_params = list(model.critic.parameters())

actor_optimizer = torch.optim.Adam(actor_params, lr=actor_lr)
critic_optimizer = torch.optim.Adam(critic_params, lr=critic_lr)

# Tracking variables
total_env_episodes = 0
max_reward = float('-inf')
batch_average_reward_history = []
batch_episode_history = []
individual_test_rewards = []
individual_test_episodes = []

Training Loop

In [ ]:
batch_reward_components_history = []
batch_episode_history = []

for batch in tqdm.tqdm(range(nb_batches)):

    # PPO Initialization
    max_possible_steps = max_timesteps_per_rollout * rollout_size

    # Temporary storage lists
    temp_states = []
    temp_actions = []
    temp_log_probs = []
    temp_state_values = []
    temp_rewards = []
    temp_is_terminal = []

    # Env Initialization
    obs, _ = env.reset()
    state_manager.update(obs)

    # Initialize target_id
    target_id = state_manager.lane_checker()
    agent_state = state_manager.ego_state_agent(target_id=target_id)

    # Episode Reset
    obs, _ = env.reset()
    state_manager.update(obs)
    done = False
    obs_old = obs
    episode_step = 0
    episode_return = 0.0
    timestep_counter = 0
    rollout = 0

    while not done and rollout < rollout_size:
        # Check for lane change trigger
        if obs[1][0] < state_manager.following_gap_threshold:
            target_id = state_manager.lane_checker()

        # Get agent state and create tensor
        agent_state = state_manager.ego_state_agent(target_id=target_id)
        state_tensor = torch.tensor(
            [agent_state[key] for key in ['vx', 'longitudinal_acceleration', 'x', 'y', 'thetha', 'lane_id', 'lane_width', 'self_curvature']],
            dtype=torch.float32, device=device
        ).unsqueeze(0)

        # Get action and value from model
        with torch.no_grad():
            agent_action, log_prob, value = model.action(state_tensor)
            action = state_manager.action(obs, agent_action)

        # Take action in environment
        obs, reward, done, truncated, info = env.step(action)
        done = done or truncated

        # Terminal conditions
        if info["crashed"] or not env.unwrapped.vehicle.on_road or timestep_counter >= max_timesteps_per_rollout:
            done = True

        # Update state manager with new observation
        state_manager.update(obs)

        # Compute reward based on the new state
        reward_components = state_manager.reward_function(obs_old, obs, target_id)
        reward = reward_components[0]

        # Store trajectory information in lists
        temp_states.append(state_tensor.squeeze(0))
        temp_actions.append(agent_action)
        temp_log_probs.append(log_prob)
        temp_state_values.append(value.squeeze())
        temp_rewards.append(torch.tensor(reward, dtype=torch.float32, device=device))
        temp_is_terminal.append(done)

        timestep_counter += 1
        obs_old = obs

        # If episode is done, increment rollout and reset for next episode
        if done:
            rollout += 1
            if rollout < rollout_size:
                obs, _ = env.reset()
                state_manager.update(obs)
                done = False
                obs_old = obs

    actual_timesteps = len(temp_states)

    buffer_states = torch.stack(temp_states)  # Shape: [actual_timesteps, state_dim]
    buffer_actions = torch.stack(temp_actions)  # Shape: [actual_timesteps, action_dim]
    buffer_log_probs = torch.stack(temp_log_probs)  # Shape: [actual_timesteps]
    buffer_state_values = torch.stack(temp_state_values)  # Shape: [actual_timesteps]
    buffer_rewards = torch.stack(temp_rewards)  # Shape: [actual_timesteps]
    buffer_is_terminal = torch.tensor(temp_is_terminal, dtype=torch.float32, device=device)  # Shape: [actual_timesteps]

    # Print rollout info
    print(f"Training Batch {batch+1}: Collected {actual_timesteps} timesteps Episode Count: {rollout}")

    # Initialize
    advantages = torch.zeros(actual_timesteps, dtype=torch.float32, device=device)
    gae = 0

    for step_idx in range(actual_timesteps-1, -1, -1):
        is_terminal = buffer_is_terminal[step_idx]

        # Determine next step values
        if step_idx == actual_timesteps - 1:
            # Last step in buffer
            next_value = 0
            next_non_terminal = 0
        else:
            next_value = buffer_state_values[step_idx + 1]
            next_non_terminal = 1 - buffer_is_terminal[step_idx]

        # TD error
        delta = buffer_rewards[step_idx] + gamma * next_value * next_non_terminal - buffer_state_values[step_idx]

        # GAE calculation
        gae = delta + gamma * gae_lambda * gae * next_non_terminal
        advantages[step_idx] = gae

        # Reset GAE AFTER processing if this step is terminal
        if is_terminal:
            gae = 0

    # More robust global normalization
    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

    # Calculate returns
    returns = advantages + buffer_state_values

    # Prepare data for policy update
    advantages_data_loader = DataLoader(
        TensorDataset(
            advantages.detach(),
            returns.detach(),              # Add pre-calculated returns
            buffer_states.detach(),
            buffer_actions.detach(),
            buffer_log_probs.detach(),
            buffer_state_values.detach()),
        batch_size=min(batch_size, actual_timesteps),
        shuffle=True)

    # Policy update (K epochs)
    for epoch in range(K):
        for batch_data in advantages_data_loader:
            b_adv, b_returns, obs_batch, action_batch, old_log_probs, old_state_values = batch_data

            # Forward pass
            dist, value = model(obs_batch)
            log_probs = dist.log_prob(action_batch)

            # Calculate ratio
            ratio = torch.exp(log_probs - old_log_probs)

            # Policy loss - use b_adv consistently
            policy_loss_1 = b_adv * ratio
            policy_loss_2 = b_adv * torch.clamp(ratio, 1-epsilon, 1+epsilon)
            policy_loss = -torch.min(policy_loss_1, policy_loss_2).mean()

            actor_loss = policy_loss - ent_coef_c2 * dist.entropy().mean()
            critic_loss = F.mse_loss(value.squeeze(-1), b_returns)  # Use pre-calculated returns

            # Optimize
            actor_optimizer.zero_grad()
            actor_loss.backward(retain_graph=True)
            torch.nn.utils.clip_grad_norm_(actor_params, max_norm=0.5)
            actor_optimizer.step()

            critic_optimizer.zero_grad()
            critic_loss.backward()
            torch.nn.utils.clip_grad_norm_(critic_params, max_norm=0.5)
            critic_optimizer.step()

    # Testing section
    if batch % test_rate == 0 and batch > 0:
        test_rewards = []
        test_reward_components = []
        frames = []

        for test_ep in range(10):
            obs, _ = env.reset()
            obs_old = obs
            episode_reward = 0
            episode_reward_components = []
            done = False
            test_step_counter = 0
            state_manager.update(obs)

            while not done and test_step_counter < 500:
                # Use state_manager lane checker for testing
                target_id = state_manager.lane_checker()
                agent_state = state_manager.ego_state_agent(target_id=target_id)

                state_tensor = torch.tensor(
                    [agent_state[key] for key in ['vx', 'longitudinal_acceleration', 'x', 'y', 'thetha', 'lane_id', 'lane_width', 'self_curvature']],
                    dtype=torch.float32, device=device
                ).unsqueeze(0)

                with torch.no_grad():
                    agent_action, log_prob, value = model.action(state_tensor, deterministic=True)

                action = state_manager.action(obs, agent_action)

                obs, _, done, truncated, info = env.step(action)
                done = done or truncated

                if test_ep % 9 == 0:
                    image = env.render()
                    frames.append(image)

                # Terminal conditions
                if info["crashed"] or not env.unwrapped.vehicle.on_road or test_step_counter >= 500:
                    done = True

                state_manager.update(obs)
                reward_components = state_manager.reward_function(obs_old, obs, target_id)

                # Store all reward components for this step
                episode_reward_components.append(reward_components)

                reward = reward_components[0]
                episode_reward += reward
                test_step_counter += 1
                obs_old = obs

            test_rewards.append(episode_reward)
            test_reward_components.append(episode_reward_components)

        # Animation code
        display(animate_image_list_efficient(frames))

        # Calculate average reward components PER TIMESTEP across all episodes
        if test_reward_components:
            # First, determine the number of reward components
            num_components = len(test_reward_components[0][0]) if test_reward_components[0] else 0

            # Calculate average per timestep for each component across all episodes
            avg_reward_components = []
            for component_idx in range(num_components):
                # Collect all timestep values for this component across all episodes
                all_timestep_values = []
                for episode_components in test_reward_components:
                    for step_components in episode_components:
                        all_timestep_values.append(step_components[component_idx])

                # Average across all timesteps from all episodes
                avg_component = sum(all_timestep_values) / len(all_timestep_values) if all_timestep_values else 0
                avg_reward_components.append(avg_component)

            # Store the average components for plotting
            if 'batch_reward_components_history' not in locals():
                batch_reward_components_history = []

            batch_reward_components_history.append(avg_reward_components)

        avg_test_reward = sum(test_rewards) / len(test_rewards)
        current_max = max(test_rewards)

        if current_max > max_reward:
            max_reward = current_max

        # Store batch info for plotting
        if 'batch_episode_history' not in locals():
            batch_episode_history = []
        batch_episode_history.append(batch)

        # Print average reward components
        if 'avg_reward_components' in locals() and avg_reward_components:
            print(f"Average Reward Components (per timestep): {[f'{comp:.3f}' for comp in avg_reward_components]}")

    # Plot every 10 batches
    if batch % 10 == 0 and batch > 0 and 'batch_reward_components_history' in locals() and batch_reward_components_history:
        plt.figure(figsize=(12, 8))

        # Convert to numpy array for easier indexing
        components_array = np.array(batch_reward_components_history)

        # Create x-axis values that match the length of components_array
        x_values = []
        for i, batch_num in enumerate(batch_episode_history):
            if batch_num % test_rate == 0 and batch_num > 0:
                x_values.append(batch_num)
                if len(x_values) == len(batch_reward_components_history):
                    break

        # Ensure we have matching lengths
        if len(x_values) != len(batch_reward_components_history):
            x_values = list(range(len(batch_reward_components_history)))

        # Define reward component names
        component_names = [
              "Total Reward",
              "Acceleration Reward (Smoothness)",
              "Rate Reward (Turn Rate)",
              "Time Reward (Lane Position)",
              "Off Road Penalty"
          ]

        # Plot each component with meaningful names
        for i in range(components_array.shape[1]):
            label = component_names[i] if i < len(component_names) else f'Component {i}'
            plt.plot(x_values,
                    components_array[:, i],
                    marker='o', linewidth=2, label=label)

        plt.title(f"Average Reward Components Per Timestep Over Training (Batch {batch})")
        plt.xlabel("Batch Number")
        plt.ylabel("Average Component Value Per Timestep")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

print("Training completed!")